# DDS Frequency Visualizer

***Collin Bradford***

Simply reads in the DDS Frequency coming from the readout block. This simply tests that the readout is working. No physical connections need to be made to the board to use this example. 

### Description of ASM_Program in DDS-Frequency.ipyng

#### Get the variables set in python
    p.memri(0,1,0,"freq")
    p.memri(0,2,5,"gain")
    p.memri(0,3,2,"nsamp")
    p.memri(1,2,3,"Nsync")
    p.memri(1,3,4,"Loop")

#### Define the number of samples and DAC settings

    p.regwi(0,4,0x9,"0b1001, stdysel = 1 (zero value), mode = 0 (nsamp), outsel = 01 (dds).")
    p.bitwi(0,4,4, "<<", 16)
    p.bitw(0,3,3,"|",4)

This value packs the four most significant bits with three parameters:
* stdysel: 
    * 0 means "after playing the waveform, keep outputting the last value"
    * 1 means "after playing the wavform, output zero"
* mode:
    * 0 means "one shot"
    * 1 means "keep on repeating the waveform"
* outsel:
    * 00 means "output is the product of table and DDS"
    * 01 means "output is coming from the DDS section"
    * 10 means "output comes from the table for the real part, and zeros for the imaginary part"
    * 11 means "output is all zeros"
    

and the lower 12 bits are `nsamp`

#### Timing and Trigger

    #Delay the start a bit
    p.synci(1000)
    #Set up trigger bits for readout and average. 
    p.regwi(1, 1, 0x4001)
    p.seti(0,1,1,0) #Trigger the readout (and average)
    p.seti(0,1,0,0) #Set the readout trigger to 0. 

The first command simply "waits" for 1000 clock ticks.  

Put a good value for triggering to page,register 1,1.

Send that value, to trigger the averaging block and the readout blocks..

The page,register of 1,0 is always a zero value, so this sends a zero to channel 0 to reset the readout block, so that it will start the next time we send a trigger.

Note that the average block, once started, just keeps on running.

#### In a loop, generate and read pulses

    #Loop
    p.label("LOOP")
    p.set(6,0,1,0,0,2,3,0)
    p.set(7,0,1,0,0,2,3,0)
    p.seti(0,1,1,0) #Trigger the readout (and average)
    p.seti(0,1,0,5) #Disable trigger 5 clocks later. 
    p.sync(1,2)
    p.loopnz(1,3,"LOOP")

Start by seetting the "LOOP" label.

Send to channel 6 the values of these registers from page 0:  1, 0, 0, 2, and 3.  Send them right away, since `time`=0.  Note that register 0 always is zero, and registers 1, 2, and 3 were set above to `freq`, `gain` and `nsamp` respectively.  This tells the signal generator to make one pulse.

Send a similar message to channel 7 to generate the same pulse on that channel.

Trigger the readout by sending the value at page,register 1,1 to channel 0 with no time delay.
Note that this was set to value 0x4001 above.

5 clock cycles later, send the value at page,register of 1,0 to channel 0.  Since the value of register 0 on all of the pages is always zero, this sets the trigger level to 0.

Wait for the number of clock cycles in page,register 1,2.  Recall that above, this was set to the value `Nsync`.

Decrement the number in the loop counter, and either loop back or continue on.

#### Clean up
    #Signal End
    p.seti(0, 1, 0, 0)
    p.end("all done")
    
Send a zero to the trigger channel, right now, to help it end well.


In [ ]:
import matplotlib.pyplot as plt

from pynq import Overlay
import numpy as np

from qsystem_2 import *
from qsystem2_asm import *

In [ ]:
#ASM 2.0 for DDS Frequency Visualization
soc = PfbSoc('qsystem_2.bit', force_init_clks=False)
with ASM_Program() as p:
    #Write all memory addresses to program memory
    p.memri(0,1,0,"freq")
    p.memri(0,2,5,"gain")
    p.memri(0,3,2,"nsamp")
    p.memri(1,2,3,"Nsync")
    p.memri(1,3,4,"Loop")
    #Set up nsamp and DAC selection register. 
    p.regwi(0,4,0x9,"0b1001, stdysel = 1 (zero value), mode = 0 (nsamp), outsel = 01 (dds).")
    p.bitwi(0,4,4, "<<", 16)
    p.bitw(0,3,3,"|",4)
    #Delay the start a bit
    p.synci(1000)
    #Set up trigger bits for readout and average. 
    p.regwi(1, 1, 0x4001)
    p.seti(0,1,1,0) #Trigger the readout (and average)
    p.seti(0,1,0,0) #Set the readout trigger to 0. 
    #Loop
    p.label("LOOP")
    p.set(6,0,1,0,0,2,3,0)
    p.set(7,0,1,0,0,2,3,0)
    p.seti(0,1,1,0) #Trigger the readout (and average)
    p.seti(0,1,0,5) #Disable trigger 5 clocks later. 
    p.sync(1,2)
    p.loopnz(1,3,"LOOP")
    #End loop
    #Signal End
    p.seti(0, 1, 0, 0)
    p.end("all done")
soc.tproc.load_asm_program(p)

In [ ]:
#Sends a pulse at the given frequency (freq) for the given time (nsamp) with the given gain (gain)
#and reads back a signal from the average buffer that is 1.3 times the duration (nsamp). 
#This is repeated (loop) times. 

debugMode = False
enablePrintSignature = True
enablePrintBuffers = False
enablePrintAverages = False
enableTestFunction = False

def getAvgSamples (freq=150, gain=30000, nsamp=150, loop=3, outputType="product"):
    
    if enablePrintSignature: 
        print(f"DAC/DDS Frequency: {freq}")
        print(f"ADC Gain: {gain}")
        print(f"Number of Samples: {nsamp}")
        print(f"Loop Count: {loop}")
        print(f"Input Type: {outputType}")
    
    #Set the address register space. 
    #Format the numbers and send them to the tproc for output frequency
    freqRegDAQ = freq2reg(soc.fs_dac, freq, B=32)
    soc.tproc.single_write(addr=0, data = freqRegDAQ)
    if debugMode: print(f"Output Frequency DAC: {freqRegDAQ}")
    #Format the nubmers and send them to the tproc for DDS frequency
    freqRegADC = freq2reg(soc.fs_adc, freq) #DDS is set to same freq as DAC
    soc.tproc.single_write(addr=1, data = freqRegADC)
    if debugMode: print(f"DDS Frequency: {freqRegADC}")
    #Simply write to the amp address WITHOUT formatting. (It is assumed that the user will do this)
    soc.tproc.single_write(addr=2, data = nsamp)
    if debugMode: print(f"nsamp: {nsamp}")
    #Write the size  of the pulse on the ADC domain (nSync). This is defined to be 1.3 times the duration (nsamp)
    nsync = int(nsamp * 1.3) #With no multiplier, there will be no delay between pulses. 
    soc.tproc.single_write(addr=3, data = nsync)
    if debugMode: print(f"nsync: {nsync}")
    #Write the loop iteration count (Loop). It should be noted that the loop iteration variable must be loopCount-1. 
    soc.tproc.single_write(addr=4, data = loop-1)
    if debugMode: print(f"loop: {loop}")
    #Write output gain WITHOUT formatting to the output gain address. 
    soc.tproc.single_write(addr=5, data = gain)
    if debugMode: print(f"gain: {gain}")
    
    #Set the size of the average buffer to nsamp (number of decimated samples) and enable the buffer. 
    soc.readouts[0].set_out(outputType)
    soc.readouts[0].set_freq(freq)
    soc.avg_bufs[0].config(address = 0, length = int(nsamp))
    soc.avg_bufs[0].enable()
    if debugMode: print(f"Average Buffer Length: {nsamp}")
    
    #Start tProc
    #soc.setSelection("product")
    soc.tproc.stop()
    soc.tproc.start()
    
    #Get I/Q decimated samples with the length loop * nsync where nsync is the number of decimated samples and 
    decimatedLength = loop * nsamp
    #idec,qdec = soc.get_decimated(ch=0, length = (loop) * nsamp)
    try:
        idec,qdec = soc.get_decimated(ch=0, length = decimatedLength)
        if enablePrintBuffers:

            #Plot everything. 
            fig,ax = plt.subplots(2, 1, sharex=True)
            ax[0].plot(idec)
            ax[0].set_ylabel("I")
            ax[1].plot(qdec)
            ax[1].set_ylabel("Q")
            ax[1].set_xlabel("Sample Number")
            ax[0].set_title("Decimated Samples")
    except RuntimeError as e:
        idec,qdec = None,None
        print("Trouble reading decimated values",e)
        
    #Get I/Q accumulated samples with the number of loop iterations. 
    try:
        iacc,qacc =  soc.get_accumulated(ch=0, length=loop)
    except RuntimeError as e:
        iacc,qacc = None,None
        print("Trouble reading accumulated values",e)
    
    if enablePrintAverages: 
        #Get the absolute value of the I and Q values
        absValues = np.sqrt(iacc ** 2 + qacc ** 2)

        #Get the phase for each sample
        phases = np.arctan2(qacc, iacc)

        #Plot everything. 
        fig,ax = plt.subplots(4,1,sharex=True)
        ax[0].plot(iacc, "*-")
        ax[0].set_ylabel("I")
        ax[1].plot(qacc, "*-")
        ax[1].set_ylabel("Q")
        ax[2].plot(absValues, "*-")
        ax[2].set_ylabel("Abs Values")
        ax[3].set_xlabel("sample number")
        ax[3].plot(phases, "*-")
        ax[3].set_ylabel("Phases")
        ax[0].set_title("from getAccumulated")
    
    return idec, qdec, iacc, qacc

In [ ]:
nsamp = 500
iterations = 2
freq = 4
outputType = "dds"

idec, qdec, iacc, qacc = getAvgSamples(freq=freq, gain=15000, nsamp=nsamp, loop=iterations, outputType=outputType)

if idec is not None:
    fig,ax = plt.subplots(2,1,sharex=True)
    ax[0].plot(idec)
    ax[0].set_title("Decimated I Samples")
    ax[0].set_ylabel("Amplitude")
    ax[1].plot(qdec)
    ax[1].set_title("Decimated Q Samples")
    ax[1].set_ylabel("Amplitdue")
    ax[1].set_xlabel("Sample")
    for i in range(0, iterations):
        ax[0].axvline(nsamp * (i+1), color="r", alpha=0.25)
        ax[1].axvline(nsamp * (i+1), color="r", alpha=0.25)
    plt.show()

if iacc is not None:
    fig,ax = plt.subplots(2,1,sharex=True)
    ax[0].plot(iacc)
    ax[0].set_title("I Accumulated Samples")
    ax[0].set_ylabel("Amplitude")
    ax[1].plot(qacc)
    ax[1].set_title("Q Accumulated Samples")
    ax[1].set_ylabel("Amplitude")
    ax[1].set_xlabel("Sample")
    plt.show()